In [1]:
from IPython.display import display, Math, Latex
from IPython.core.display import HTML 

In [19]:
# подгрузим модули
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [50]:
# загрузим данные
dataset = pd.read_csv('data/Market_Basket_Optimisation.csv', header = None)

In [51]:
# посомтрим на датасет
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
dataset.fillna(method = 'ffill',axis = 1, inplace = True)

Видим, что датасет у нас представляет разреженную матрицу, где в строках у нас набор items в каждой транзакции.

In [23]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs
2,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney
3,turkey,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado
4,mineral water,milk,energy bar,whole wheat rice,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea


In [24]:
#создаим из них матрицу
transactions = []
for i in range(0, 7501): 
    transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])

In [56]:
#загружаем apriori
from apyori import apriori

In [57]:
%%time
# и обучимся правилам. Обратите внимание, что пороговые значения мы вибираем сами в зависимости от того, /
# насколкьо "сильные" правила мы хотим получить
# min_support -- минимальный support для правил (dtype = float).
# min_confidence -- минимальное значение confidence для правил (dtype = float)
# min_lift -- минимальный lift (dtype = float)
# max_length -- максимальная длина itemset (вспоминаем про k-itemset)  (dtype = integer)

result = list(apyori.apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 4, min_length = 2))

Wall time: 664 ms


Визуализируем выход

In [27]:
import shutil, os 

In [28]:
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [29]:
import json #преобразовывать будем в json, используя встроенные в модуль методы

In [30]:
output = []
for RelationRecord in result:
    o = StringIO()
    apyori.dump_as_json(RelationRecord, o)
    output.append(json.loads(o.getvalue()))
data_df = pd.DataFrame(output)

In [31]:
# и взгялнем на итоги
pd.set_option('display.max_colwidth', -1)

from IPython.display import display, HTML

display(HTML(data_df.to_html()))

,items,ordered_statistics,support
0,"[chicken, light cream]","[{'items_base': ['light cream'], 'items_add': ['chicken'], 'confidence': 0.29059829059829057, 'lift': 4.84395061728395}]",0.004533
1,"[escalope, pasta]","[{'items_base': ['pasta'], 'items_add': ['escalope'], 'confidence': 0.3728813559322034, 'lift': 4.700811850163794}]",0.005866
2,"[fromage blanc, honey]","[{'items_base': ['fromage blanc'], 'items_add': ['honey'], 'confidence': 0.2450980392156863, 'lift': 5.164270764485569}]",0.003333
3,"[olive oil, whole wheat pasta]","[{'items_base': ['whole wheat pasta'], 'items_add': ['olive oil'], 'confidence': 0.2714932126696833, 'lift': 4.122410097642296}]",0.007999
4,"[pasta, shrimp]","[{'items_base': ['pasta'], 'items_add': ['shrimp'], 'confidence': 0.3220338983050847, 'lift': 4.506672147735896}]",0.005066
5,"[cake, frozen vegetables, tomatoes]","[{'items_base': ['cake', 'frozen vegetables'], 'items_add': ['tomatoes'], 'confidence': 0.2987012987012987, 'lift': 4.367560314928736}]",0.003066
6,"[cereals, ground beef, spaghetti]","[{'items_base': ['cereals', 'spaghetti'], 'items_add': ['ground beef'], 'confidence': 0.45999999999999996, 'lift': 4.681763907734057}]",0.003066
7,"[chocolate, ground beef, herb & pepper]","[{'items_base': ['chocolate', 'herb & pepper'], 'items_add': ['ground beef'], 'confidence': 0.4411764705882354, 'lift': 4.4901827759597746}]",0.003999
8,"[eggs, ground beef, herb & pepper]","[{'items_base': ['eggs', 'ground beef'], 'items_add': ['herb & pepper'], 'confidence': 0.2066666666666667, 'lift': 4.178454627133872}]",0.004133
9,"[french fries, ground beef, herb & pepper]","[{'items_base': ['french fries', 'ground beef'], 'items_add': ['herb & pepper'], 'confidence': 0.23076923076923078, 'lift': 4.665768194070081}, {'items_base': ['french fries', 'herb & pepper'], 'items_add': ['ground beef'], 'confidence': 0.46153846153846156, 'lift': 4.697421981004071}]",0.003200


Итого мы видим:

1. Пары items
2. items_base - первый элемент пары
3. items_add - второй (добавленный алгоритмом) элемент пары
4. confidence - значение confidence для пары
5. lift - значение lift для пары
6. support - начение support для пары. При желании, по нему можно отсортировать 


Результаты логичные: эскалоп и макароны, эскалоп и сливочно-грибной соус, курица и нежирная сметана, мягкий сыр и мед и т.д. - все это вполне логичные и, главное, вкусные сочетания:)

In [52]:
import numpy as np
"""
Класс инициируется 3мя параметрами:
- min_supp - минимальный support  который мы рассматриваем для ItemSet. Рассчитывается как % от количества транзакций
- max_items - максимальное количество елементов в нашем ItemSet
- min_items - минимальное количество элементов ItemSet
"""
class Eclat:
    #инициализация объекта класса
    def __init__(self, min_support = 0.01, max_items = 5, min_items = 2):
        self.min_support = min_support
        self.max_items = max_items
        self.min_items = min_items
        self.item_lst = list()
        self.item_len = 0
        self.item_dict = dict()
        self.final_dict = dict()
        self.data_size = 0
    
    #создание словаря из ненулевых объектов из всех транзакций (вертикальный датасет)
    def read_data(self, dataset):
        for index, row in dataset.iterrows():
            row_wo_na = row.dropna().unique()
            for item in row_wo_na:
                item = item.strip()
                if item in self.item_dict:
                    self.item_dict[item][0] += 1
                else:
                    self.item_dict.setdefault(item, []).append(1)
                self.item_dict[item].append(index)
        #задаем переменные экземпляра (instance variables)
        self.data_size = dataset.shape[0]
        self.item_lst = list(self.item_dict.keys())
        self.item_len = len(self.item_lst)
        self.min_support = self.min_support * self.data_size
        #print ("min_supp", self.min_support)
        
    #рекурсивный метод для поиска всех ItemSet по алгоритму Eclat
    #структура данных: {Item: [Supp number, tid1, tid2, tid3, ...]}
    def recur_eclat(self, item_name, tids_array, minsupp, num_items, k_start):
        if tids_array[0] >= minsupp and num_items <= self.max_items:
            for k in range(k_start+1, self.item_len):
                if self.item_dict[self.item_lst[k]][0] >= minsupp:
                    new_item = item_name + " | " + self.item_lst[k]
                    new_tids = np.intersect1d(tids_array[1:], self.item_dict[self.item_lst[k]][1:])
                    new_tids_size = new_tids.size
                    new_tids = np.insert(new_tids, 0, new_tids_size)
                    if new_tids_size >= minsupp:
                        if num_items >= self.min_items: self.final_dict.update({new_item: new_tids})
                        self.recur_eclat(new_item, new_tids, minsupp, num_items+1, k)
    
    #последовательный вызов функций определенных выше
    def fit(self, dataset):
        i = 0
        self.read_data(dataset)
        for w in self.item_lst:
            self.recur_eclat(w, self.item_dict[w], self.min_support, 2, i)
            i+=1
        return self
        
    #вывод в форме словаря {ItemSet: support(ItemSet)}
    def transform(self):
        return {k: "{0:.4f}%".format((v[0]+0.0)/self.data_size*100) for k, v in self.final_dict.items()}

Потестируем

In [53]:
#создадим экземпляр класса с нужными нам параметрами
model = Eclat(min_support = 0.01, max_items = 4, min_items = 3)

In [54]:
#обучим
model.fit(dataset)

In [55]:
#и визуализируем результаты
model.transform()

{'eggs | spaghetti | chocolate': '1.0532%',
 'milk | spaghetti | chocolate': '1.0932%',
 'mineral water | chocolate | ground beef': '1.0932%',
 'mineral water | eggs | chocolate': '1.3465%',
 'mineral water | eggs | ground beef': '1.0132%',
 'mineral water | eggs | milk': '1.3065%',
 'mineral water | eggs | spaghetti': '1.4265%',
 'mineral water | french fries | spaghetti': '1.0132%',
 'mineral water | frozen vegetables | spaghetti': '1.1998%',
 'mineral water | milk | chocolate': '1.3998%',
 'mineral water | milk | frozen vegetables': '1.1065%',
 'mineral water | milk | ground beef': '1.1065%',
 'mineral water | milk | spaghetti': '1.5731%',
 'mineral water | olive oil | spaghetti': '1.0265%',
 'mineral water | spaghetti | chocolate': '1.5865%',
 'mineral water | spaghetti | ground beef': '1.7064%',
 'mineral water | spaghetti | pancakes': '1.1465%'}

Как видно, реализовать алгоритм своими силами довольно просто, хотя с эффективностью стоит поработать:)

In [32]:
#!pip install pyfpgrowth - установка

In [33]:
import pyfpgrowth

In [34]:
#Сгенериуем паттерны
patterns = pyfpgrowth.find_frequent_patterns(transactions, 2)

Wall time: 1min 38s


In [ ]:
#Выучим правила
rules = pyfpgrowth.generate_association_rules(patterns, 30);

In [ ]:
#Покажем
rules;